# 第13章 RAG

## 13.2 基本的な RAG のシステムの実装

### 13.2.2 LangChain で LLM と文埋め込みモデルを使う

#### 環境の準備

In [ ]:
!pip install transformers[torch,sentencepiece] langchain langchain-community langchain-huggingface faiss-cpu jq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 4.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.

In [ ]:
from huggingface_hub import notebook_login

# Hugging Face Hubにログイン
notebook_login()

In [ ]:
from transformers.trainer_utils import set_seed

# 乱数のシードを設定
set_seed(42)

#### LangChain で LLM を使う

In [ ]:
import torch
from langchain_huggingface import HuggingFacePipeline
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    pipeline,
)

# Hugging Face Hubにおけるモデル名を指定
model_name = "llm-book/Swallow-7b-hf-oasst1-21k-ja"

# モデルを読み込む
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

# トークナイザを読み込む
tokenizer = AutoTokenizer.from_pretrained(model_name)

# テキスト生成用のパラメータを指定
generation_config = {
    "max_new_tokens": 128,
    "do_sample": False,
    "temperature": None,
    "top_p": None,
}

# テキスト生成を行うパイプラインを作成
text_generation_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device_map="auto",
    **generation_config,
)

# パイプラインからLangChainのLLMコンポーネントを作成
llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

config.json:   0%|          | 0.00/761 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/3.77G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/183 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/914k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.30M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/548 [00:00<?, ?B/s]

In [ ]:
from pprint import pprint

# モデルに入力する会話データ
llm_prompt_messages = [
    {"role": "user", "content": "四国地方で一番高い山は？"},
]

# 会話データにチャットテンプレートを適用し、内容を確認
llm_prompt_text = tokenizer.apply_chat_template(
    llm_prompt_messages,
    tokenize=False,
    add_generation_prompt=True,
)
print(llm_prompt_text)

<s>ユーザ：四国地方で一番高い山は？</s><s>アシスタント：


In [ ]:
# LLMへの入力を実行し、結果を確認
llm_output_message = llm.invoke(llm_prompt_text)
print(llm_output_message)

<s>ユーザ：四国地方で一番高い山は？</s><s>アシスタント：日本の四国地方で最も高い山は、徳島県と高知県の県境に位置する剣山（つるぎさん、1,955m）である。剣山は四国の最高峰であり、日本の百名山のひとつである。


#### Chat Model コンポーネントの利用

In [ ]:
from langchain_huggingface import ChatHuggingFace

# LLMコンポーネントからChat Modelコンポーネントを作成
chat_model = ChatHuggingFace(llm=llm, tokenizer=tokenizer)

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage

# Chat Modelに入力する会話データ
chat_messages = [HumanMessage(content="四国地方で一番高い山は？")]

# Chat Modelによるチャットテンプレート適用後の入力文字列を確認
chat_prompt = chat_model._to_chat_prompt(chat_messages)
print(chat_prompt)

<s>ユーザ：四国地方で一番高い山は？</s><s>アシスタント：


In [ ]:
# Chat Modelに会話データを入力し、出力を確認
chat_output_message = chat_model.invoke(chat_messages)
pprint(chat_output_message)

AIMessage(content='<s>ユーザ：四国地方で一番高い山は？</s><s>アシスタント：日本の四国地方で最も高い山は、徳島県と高知県の県境に位置する剣山（つるぎさん、1,955m）である。剣山は四国の最高峰であり、日本の百名山のひとつである。', id='run-62e12026-9cda-4d2c-a962-ccee9e3df9bc-0')


In [ ]:
# Chat Modelが出力したテキストからモデルの応答部分のみを抽出
response_text = chat_output_message.content[len(chat_prompt) :]
print(response_text)

日本の四国地方で最も高い山は、徳島県と高知県の県境に位置する剣山（つるぎさん、1,955m）である。剣山は四国の最高峰であり、日本の百名山のひとつである。


#### Chain を構築する

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

# 任意のqueryからプロンプトを構築するPrompt Templateを作成
prompt_template = ChatPromptTemplate.from_messages(
    [("user", "{query}")]
)

# Prompt Templateを実行し、結果を確認
prompt_template_output = prompt_template.invoke(
    {"query": "四国地方で一番高い山は？"}
)
pprint(prompt_template_output)

ChatPromptValue(messages=[HumanMessage(content='四国地方で一番高い山は？')])


In [ ]:
# Prompt TemplateとChat Modelを連結したChainを作成
chain = prompt_template | chat_model

# Chainを実行し、結果を確認
chain_output = chain.invoke({"query": "四国地方で一番高い山は？"})
pprint(chain_output)

AIMessage(content='<s>ユーザ：四国地方で一番高い山は？</s><s>アシスタント：日本の四国地方で最も高い山は、徳島県と高知県の県境に位置する剣山（つるぎさん、1,955m）である。剣山は四国の最高峰であり、日本の百名山のひとつである。', id='run-2cbc476a-022c-46a4-81e1-1754cfd61bb4-0')


In [ ]:
from langchain_core.prompt_values import ChatPromptValue
from langchain_core.runnables import RunnableLambda

def chat_model_resp_only_func(
    chat_prompt_value: ChatPromptValue,
) -> str:
    """chat_modelにchat_prompt_valueを入力し、
    出力からモデルの応答部分のみを文字列で返す"""
    chat_prompt = chat_model._to_chat_prompt(
        chat_prompt_value.messages
    )
    chat_output_message = chat_model.invoke(chat_prompt_value)
    response_text = chat_output_message.content[len(chat_prompt) :]
    return response_text

# 定義した関数の処理を行うRunnableを作成
chat_model_resp_only = RunnableLambda(chat_model_resp_only_func)

# Prompt TemplateとRunnableを連結したChainを作成
chain_resp_only = prompt_template | chat_model_resp_only

# Chainを実行し、結果を確認
chain_resp_only_output = chain_resp_only.invoke(
    {"query": "四国地方で一番高い山は？"}
)
print(chain_resp_only_output)

日本の四国地方で最も高い山は、徳島県と高知県の県境に位置する剣山（つるぎさん、1,955m）である。剣山は四国の最高峰であり、日本の百名山のひとつである。


#### LangChain で文埋め込みモデルを使う


In [ ]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

# Hugging Face Hubにおけるモデル名を指定
embedding_model_name = "BAAI/bge-m3"

# モデル名からEmbedding Modelを初期化
embedding_model = HuggingFaceEmbeddings(
    model_name=embedding_model_name,
    model_kwargs={"model_kwargs": {"torch_dtype": torch.float16}},
)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/15.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/687 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [ ]:
sample_texts = [
    "日本で一番高い山は何ですか？",
    "日本で一番高い山は富士山です。",
]

# 二つのテキストに対して文埋め込みを実行し、結果を確認
sample_embeddings = embedding_model.embed_documents(sample_texts)
print(sample_embeddings)

[[0.01058197021484375, 0.032470703125, -0.024261474609375, -0.0253448486328125, 0.01114654541015625, -0.030426025390625, -0.01139068603515625, -0.0119476318359375, -0.034393310546875, -0.026214599609375, -0.0206298828125, 0.0271148681640625, -0.0203399658203125, 0.00777435302734375, 0.042205810546875, -0.027557373046875, 0.042449951171875, 0.0017147064208984375, -0.0020751953125, 0.023223876953125, -0.005340576171875, -0.0212249755859375, -0.0307769775390625, 0.030609130859375, 0.00620269775390625, 0.0177154541015625, -0.01161956787109375, 0.004650115966796875, -0.005558013916015625, -0.037994384765625, -0.0236053466796875, -0.02044677734375, 0.0025005340576171875, -0.0026092529296875, -0.0281219482421875, -0.00820159912109375, -0.00020563602447509766, -0.0306243896484375, -0.03692626953125, 0.001079559326171875, 0.034149169921875, 0.0195465087890625, 0.0406494140625, -0.0242767333984375, -0.043792724609375, -0.0181427001953125, -0.019256591796875, -0.0309600830078125, 0.01884460449218

In [ ]:
# 二つのテキストの文埋め込みから類似度を計算
similarity = torch.nn.functional.cosine_similarity(
    torch.tensor([sample_embeddings[0]]),
    torch.tensor([sample_embeddings[1]]),
)
print(similarity)

tensor([0.7743])


### 13.2.3 LangChain で RAG を実装する

#### データストアの構築



In [ ]:
# 検索対象の文書集合のファイルをダウンロード
!wget \
https://github.com/ghmagazine/llm-book/raw/main/chapter13/docs.json

In [ ]:
from langchain_community.document_loaders import JSONLoader

# JSONファイルから文書を読み込むためのDocument Loaderを初期化
document_loader = JSONLoader(
    file_path="./docs.json",  # 読み込みを行うファイル
    jq_schema=".text",  # 読み込み対象のフィールド
    json_lines=True,  # JSON Lines形式のファイルであることを指定
)

# 文書の読み込みを実行
documents = document_loader.load()

# 読み込まれた文書数を確認
print(len(documents))

103


In [ ]:
# 読み込まれた文書の内容を確認
pprint(documents[0])

Document(metadata={'source': '/content/docs.json', 'seq_num': 1}, page_content='富士山（ふじさん）は、静岡県（富士宮市、富士市、裾野市、御殿場市、駿東郡小山町）と山梨県（富士吉田市、南都留郡鳴沢村）に跨る活火山である。標高3776.12 m、日本最高峰（剣ヶ峰）の独立峰で、その優美な風貌は日本国外でも日本の象徴として広く知られている。 数多くの芸術作品の題材とされ芸術面のみならず、気候や地層など地質学的にも社会に大きな影響を与えている。懸垂曲線の山容を有した玄武岩質成層火山で構成され、その山体は駿河湾の海岸まで及ぶ。 古来より霊峰とされ、特に山頂部は浅間大神が鎮座するとされたため、神聖視された。噴火を沈静化するため律令国家により浅間神社が祭祀され、浅間信仰が確立された。また、富士山修験道の開祖とされる富士上人により修験道の霊場としても認識されるようになり、登拝が行われるようになった。これら富士信仰は時代により多様化し、村山修験や富士講といった一派を形成するに至る。現在、富士山麓周辺には観光名所が多くある他、夏季シーズンには富士登山が盛んである。 日本三名山（三霊山）、日本百名山、日本の地質百選に選定されている。また、1936年（昭和11年）には富士箱根伊豆国立公園に指定されている。その後、1952年（昭和27年）に特別名勝、2011年（平成23年）に史跡、さらに2013年（平成25年）6月22日には関連する文化財群とともに「富士山-信仰の対象と芸術の源泉」の名で世界文化遺産に登録された。 富士山についての最も古い記録は『常陸国風土記』における「福慈岳」という語であると言われている。他にも多くの呼称が存在し、不二山もしくは不尽山と表記する古文献もある。また、『竹取物語』における伝説もある。「フジ」という長い山の斜面を表す大和言葉から転じて富士山と称されたという説もある。近代以降の語源説としては、宣教師バチェラーは、名前は「火を噴く山」を意味するアイヌ語の「フンチヌプリ」に由来するとの説を提示した。しかし、これは囲炉裏の中に鎮座する火の姥神を表す「アペフチカムイ」からきた誤解であるとの反論がある。その他の語源説として、マレー語説、マオリ語説、原ポリネシア語説がある。 明確に「富士山」と表記さ

In [ ]:
# 読み込まれた文書の長さ（文字数）を確認
print(len(documents[0].page_content))

21232


In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 文書を指定した文字数で分割するText Splitterを初期化
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=400,  # 分割する最大文字数
    chunk_overlap=100,  # 分割された文書間で重複させる最大文字数
    add_start_index=True,  # 元の文書における開始位置の情報を付与
)

# 文書の分割を実行
split_documents = text_splitter.split_documents(documents)

# 分割後の文書数を確認
print(len(split_documents))

1475


In [ ]:
# 分割後の文書の内容を確認
pprint(split_documents[0])
pprint(split_documents[1])

Document(metadata={'source': '/content/docs.json', 'seq_num': 1, 'start_index': 0}, page_content='富士山（ふじさん）は、静岡県（富士宮市、富士市、裾野市、御殿場市、駿東郡小山町）と山梨県（富士吉田市、南都留郡鳴沢村）に跨る活火山である。標高3776.12 m、日本最高峰（剣ヶ峰）の独立峰で、その優美な風貌は日本国外でも日本の象徴として広く知られている。 数多くの芸術作品の題材とされ芸術面のみならず、気候や地層など地質学的にも社会に大きな影響を与えている。懸垂曲線の山容を有した玄武岩質成層火山で構成され、その山体は駿河湾の海岸まで及ぶ。')
Document(metadata={'source': '/content/docs.json', 'seq_num': 1, 'start_index': 129}, page_content='数多くの芸術作品の題材とされ芸術面のみならず、気候や地層など地質学的にも社会に大きな影響を与えている。懸垂曲線の山容を有した玄武岩質成層火山で構成され、その山体は駿河湾の海岸まで及ぶ。 古来より霊峰とされ、特に山頂部は浅間大神が鎮座するとされたため、神聖視された。噴火を沈静化するため律令国家により浅間神社が祭祀され、浅間信仰が確立された。また、富士山修験道の開祖とされる富士上人により修験道の霊場としても認識されるようになり、登拝が行われるようになった。これら富士信仰は時代により多様化し、村山修験や富士講といった一派を形成するに至る。現在、富士山麓周辺には観光名所が多くある他、夏季シーズンには富士登山が盛んである。')


In [ ]:
# 分割後の文書の長さ（文字数）を確認
print(len(split_documents[0].page_content))
print(len(split_documents[1].page_content))

221
310


#### ベクトルインデックスの作成

In [ ]:
from langchain_community.vectorstores import FAISS

# 分割後の文書と文埋め込みモデルを用いて、Faissのベクトルインデックスを作成
vectorstore = FAISS.from_documents(split_documents, embedding_model)

# ベクトルインデックスに登録された文書数を確認
print(vectorstore.index.ntotal)

1475


#### Retriever コンポーネントの作成

In [ ]:
# ベクトルインデックスを元に文書の検索を行うRetrieverを初期化
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

In [ ]:
# 文書の検索を実行
retrieved_documents = retriever.invoke("四国地方で一番高い山は？")

# 検索された文書を確認
pprint(retrieved_documents)

[Document(metadata={'source': '/content/docs.json', 'seq_num': 26, 'start_index': 0}, page_content='この項目に含まれる文字「鎚」は、オペレーティングシステムやブラウザなどの環境により表示が異なります。 石鎚山（いしづちさん、いしづちやま）は、四国山地西部に位置する標高1,982 mの山で、近畿以西を「西日本」とした場合の西日本最高峰で、山頂から望む展望が四国八十八景64番に選定。愛媛県西条市と久万高原町の境界に位置する。 石鉄山、石鈇山、石土山、石槌山とも表記され、伊予の高嶺とも呼ばれる。『日本霊異記』には「石槌山」と記され、延喜式の神名帳（延喜式神名帳）では「石鉄神社」と記されている。前神寺および横峰寺では「石鈇山（しゃくまざん）」とも呼ぶ。'),
 Document(metadata={'source': '/content/docs.json', 'seq_num': 1, 'start_index': 0}, page_content='富士山（ふじさん）は、静岡県（富士宮市、富士市、裾野市、御殿場市、駿東郡小山町）と山梨県（富士吉田市、南都留郡鳴沢村）に跨る活火山である。標高3776.12 m、日本最高峰（剣ヶ峰）の独立峰で、その優美な風貌は日本国外でも日本の象徴として広く知られている。 数多くの芸術作品の題材とされ芸術面のみならず、気候や地層など地質学的にも社会に大きな影響を与えている。懸垂曲線の山容を有した玄武岩質成層火山で構成され、その山体は駿河湾の海岸まで及ぶ。'),
 Document(metadata={'source': '/content/docs.json', 'seq_num': 96, 'start_index': 0}, page_content='四阿山（あずまやさん）は、長野県と群馬県の県境に跨る山。標高2,354 m。日本百名山の一つに数えられている。吾妻山・吾嬬山（あがつまやま）などとも呼ばれ、嬬恋村では吾妻山が用いられている。 上信国境の山では、浅間山 (2,568m) に次ぐ標高であり志賀高原最高峰、裏岩菅山 (2,341m) より13m高いが、東北最高峰である燧ヶ岳 (2,356m) より2m低い。 約80万年前から30

#### RAG の Chain の構築

In [ ]:
# 任意のqueryからメッセージを構築するPrompt Templateを作成
rag_prompt_text = (
    "以下の文書の内容を参考にして、質問に答えてください。\n\n"
    "---\n{context}\n---\n\n質問: {query}"
)
rag_prompt_template = ChatPromptTemplate.from_messages(
    [("user", rag_prompt_text)]
)

In [ ]:
from langchain_core.documents import Document

def format_documents_func(documents: list[Document]) -> str:
    """文書のリストを改行で連結した一つの文字列として返す"""
    return "\n\n".join(
        document.page_content for document in documents
    )

# 定義した関数の処理を行うRunnableを作成
format_documents = RunnableLambda(format_documents_func)

In [ ]:
from langchain_core.runnables import RunnablePassthrough

# RAGの一連の処理を行うChainを作成
rag_chain = (
    {
        "context": retriever | format_documents,
        "query": RunnablePassthrough(),
    }
    | rag_prompt_template
    | chat_model_resp_only
)

In [ ]:
# Chainを実行し、結果を確認
rag_chain_output = rag_chain.invoke("四国地方で一番高い山は？")
print(rag_chain_output)

四国地方で一番高い山は、愛媛県と高知県の県境にある石鎚山です。標高は1,982メートルで、四国地方で最も高い山です。
